In this assignment, you'll continue working with the house prices data. Compare your solution to [this example solution](https://github.com/Thinkful-Ed/machine-learning-regression-problems/blob/master/notebooks/7.solution_overfitting_and_regularization.ipynb).

* Load the **houseprices** data from Thinkful's database.
* Reimplement your model from the previous checkpoint.
* Try OLS, Lasso, Ridge, and ElasticNet regression using the same model specification. This time, you need to do **k-fold cross-validation** to choose the best hyperparameter values for your models. Scikit-learn has RidgeCV, LassoCV, and ElasticNetCV that you can utilize to do this. Which model is the best? Why?

In [19]:
######################################
######   Notice the new imports  #####
###### Ridge, Lasso and ElasticNet ###
######################################

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.linear_model import LinearRegression, RidgeCV, LassoCV, ElasticNetCV
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
import statsmodels.api as sm
from statsmodels.tools.eval_measures import mse, rmse
from sqlalchemy import create_engine

import warnings
warnings.filterwarnings('ignore')

postgres_user = 'dsbc_student'
postgres_pw = '7*.8G9QH21'
postgres_host = '142.93.121.174'
postgres_port = '5432'
postgres_db = 'houseprices'

engine = create_engine('postgresql://{}:{}@{}:{}/{}'.format(
    postgres_user, postgres_pw, postgres_host, postgres_port, postgres_db))
houses_df = pd.read_sql_query('select * from houseprices',con=engine)

# no need for an open connection, as we're only doing a single query
engine.dispose()

houses_df.head(3)

,id,mssubclass,mszoning,lotfrontage,lotarea,street,alley,lotshape,landcontour,utilities,...,poolarea,poolqc,fence,miscfeature,miscval,mosold,yrsold,saletype,salecondition,saleprice
0,1,60,RL,65.0,8450,Pave,None,Reg,Lvl,AllPub,...,0,None,None,None,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,None,Reg,Lvl,AllPub,...,0,None,None,None,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,None,IR1,Lvl,AllPub,...,0,None,None,None,0,9,2008,WD,Normal,223500


In [8]:
# Replace missing numerical values with the mean and missing categoricals ('None') with 'No--' 
# -- = first to letters of feature, ie. NoFe = 'no fence' 
# and the lone missing electrical value with house_df.electrical.mode()

houses_df['electrical'].fillna(str(houses_df['electrical'].mode()), inplace=True)

for column_name in houses_df.columns[3:]:
    for idx, value in enumerate(houses_df[column_name]):
        if pd.isna(value):
            try:
                houses_df[column_name].fillna(houses_df[column_name].mean(), inplace=True)
            except:
                houses_df[column_name].fillna('No' + column_name[:2].capitalize(), inplace=True)

In [9]:
# Creating our dummy variables
columns = ['alley','poolqc','garagetype']

# Create a dataframe with added dummy features
houses_df_dummies = pd.get_dummies(houses_df, columns=columns, drop_first=True)
dummy_columns = [c for c in houses_df_dummies.columns if any([c.startswith(n) for n in columns])]
dummy_columns

['alley_NoAl',
 'alley_Pave',
 'poolqc_Fa',
 'poolqc_Gd',
 'poolqc_NoPo',
 'garagetype_Attchd',
 'garagetype_Basment',
 'garagetype_BuiltIn',
 'garagetype_CarPort',
 'garagetype_Detchd',
 'garagetype_NoGa']

In [22]:
# Using the features of our most successful model so far (adjusted R-squared of 0.738)
features = ['alley_Pave','poolqc_Fa','poolqc_Gd','poolqc_NoPo','garagetype_Attchd','garagetype_BuiltIn', 'grlivarea',
                'garagetype_NoGa','lotarea','totalbsmtsf','garagearea', 'bedroomabvgr','yearbuilt','fullbath']

X = houses_df_dummies[features]
Y = houses_df_dummies.saleprice

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size = 0.2, random_state = 465)

# Don't forget to add a constant
X_train = sm.add_constant(X_train)

# Let's train our new model
results = sm.OLS(y_train, X_train).fit()
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              saleprice   R-squared:                       0.740
Model:                            OLS   Adj. R-squared:                  0.738
Method:                 Least Squares   F-statistic:                     253.3
Date:                Thu, 30 Jan 2020   Prob (F-statistic):               0.00
Time:                        21:29:55   Log-Likelihood:                -14038.
No. Observations:                1168   AIC:                         2.810e+04
Df Residuals:                    1154   BIC:                         2.817e+04
Df Model:                          13                                         
Covariance Type:            nonrobust                                         
======================================================================================
                         coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------
const              -1.202e+06   1.21e+05     -9.962      0.000   -1.44e+06   -9.65e+05
alley_Pave         -2742.8293   7324.308     -0.374      0.708   -1.71e+04    1.16e+04
poolqc_Fa          -1.293e-06   1.19e-07    -10.840      0.000   -1.53e-06   -1.06e-06
poolqc_Gd          -2.431e+05      5e+04     -4.858      0.000   -3.41e+05   -1.45e+05
poolqc_NoPo          6.17e+04   4.07e+04      1.515      0.130   -1.82e+04    1.42e+05
garagetype_Attchd   9286.0806   3315.988      2.800      0.005    2780.039    1.58e+04
garagetype_BuiltIn  1.893e+04   6227.183      3.040      0.002    6712.289    3.11e+04
grlivarea             80.6000      3.895     20.694      0.000      72.958      88.242
garagetype_NoGa     1.321e+04   6569.288      2.011      0.045     319.608    2.61e+04
lotarea                0.4380      0.117      3.741      0.000       0.208       0.668
totalbsmtsf           34.2990      3.534      9.706      0.000      27.365      41.233
garagearea            65.8377      8.418      7.821      0.000      49.321      82.354
bedroomabvgr       -1.081e+04   1822.168     -5.933      0.000   -1.44e+04   -7235.123
yearbuilt            584.1647     59.421      9.831      0.000     467.579     700.751
fullbath            -580.2923   3191.278     -0.182      0.856   -6841.650    5681.066
==============================================================================
Omnibus:                      308.034   Durbin-Watson:                   1.875
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            18164.468
Skew:                           0.237   Prob(JB):                         0.00
Kurtosis:                      22.314   Cond. No.                     6.52e+17
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The smallest eigenvalue is 6.39e-25. This might indicate that there are
strong multicollinearity problems or that the design matrix is singular.
"""

In [67]:
# Using, and comparing sklearn's regression models, First, OLS...
# Undoing the sm.add_constant() used with stats model's OLS algorithm
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size = 0.2, random_state = 465)

lrm = LinearRegression()

lrm.fit(X_train, y_train)

# We are making predictions here
y_preds_train = lrm.predict(X_train)
y_preds_test = lrm.predict(X_test)

print("R-squared of the model in training set is: {}".format(lrm.score(X_train, y_train)))
print("Mean absolute percentage error of the prediction is: {}".format(np.mean(np.abs((y_train - y_preds_train) / y_train)) * 100))
print("-----Test set statistics-----")
print("R-squared of the model in test set is: {}".format(lrm.score(X_test, y_test)))
print("Mean absolute error of the prediction is: {}".format(mean_absolute_error(y_test, y_preds_test)))
print("Mean squared error of the prediction is: {}".format(mse(y_test, y_preds_test)))
print("Root mean squared error of the prediction is: {}".format(rmse(y_test, y_preds_test)))
print("Mean absolute percentage error of the prediction is: {}".format(np.mean(np.abs((y_test - y_preds_test) / y_test)) * 100))

R-squared of the model in training set is: 0.7404898794013047
Mean absolute percentage error of the prediction is: 14.919336949951001
-----Test set statistics-----
R-squared of the model in test set is: 0.6998512505650885
Mean absolute error of the prediction is: 25488.835282763495
Mean squared error of the prediction is: 2015109883.409951
Root mean squared error of the prediction is: 44889.97531086368
Mean absolute percentage error of the prediction is: 14.492972032862902


In [68]:
alphas = [np.power(10.0,p) for p in np.arange(-10,40,1)]

# Fitting a ridge regression model with cross valivation. Alpha is the regularization parameter 
# (usually called lambda). As alpha gets larger, parameter shrinkage grows more pronounced.
ridge_cv = RidgeCV(alphas=alphas, cv=10)

ridge_cv.fit(X_train, y_train)

# We are making predictions here
y_preds_train = ridge_cv.predict(X_train)
y_preds_test = ridge_cv.predict(X_test)

print("Best alpha value is: {}".format(ridge_cv.alpha_))
print("R-squared of the model in training set is: {}".format(ridge_cv.score(X_train, y_train)))
print("Mean absolute percentage error of the prediction is: {}".format(np.mean(np.abs((y_train - y_preds_train) / y_train)) * 100))
print("-----Test set statistics-----")
print("R-squared of the model in test set is: {}".format(ridge_cv.score(X_test, y_test)))
print("Mean absolute error of the prediction is: {}".format(mean_absolute_error(y_test, y_preds_test)))
print("Mean squared error of the prediction is: {}".format(mse(y_test, y_preds_test)))
print("Root mean squared error of the prediction is: {}".format(rmse(y_test, y_preds_test)))
print("Mean absolute percentage error of the prediction is: {}".format(np.mean(np.abs((y_test - y_preds_test) / y_test)) * 100))

Best alpha value is: 100.0
R-squared of the model in training set is: 0.716152613470908
Mean absolute percentage error of the prediction is: 15.017776154952905
-----Test set statistics-----
R-squared of the model in test set is: 0.7693304875725617
Mean absolute error of the prediction is: 24514.246218534543
Mean squared error of the prediction is: 1548646846.5019689
Root mean squared error of the prediction is: 39352.850551160445
Mean absolute percentage error of the prediction is: 13.977760381986007


In [69]:
# Fitting a LASSO regression model with cross valivation.
lasso_cv = LassoCV(alphas=alphas, cv=10)

lasso_cv.fit(X_train, y_train)

# We are making predictions here
y_preds_train = lasso_cv.predict(X_train)
y_preds_test = lasso_cv.predict(X_test)

print("Best alpha value is: {}".format(lasso_cv.alpha_))
print("R-squared of the model in training set is: {}".format(lasso_cv.score(X_train, y_train)))
print("Mean absolute percentage error of the prediction is: {}".format(np.mean(np.abs((y_train - y_preds_train) / y_train)) * 100))
print("-----Test set statistics-----")
print("R-squared of the model in test set is: {}".format(lasso_cv.score(X_test, y_test)))
print("Mean absolute error of the prediction is: {}".format(mean_absolute_error(y_test, y_preds_test)))
print("Mean squared error of the prediction is: {}".format(mse(y_test, y_preds_test)))
print("Root mean squared error of the prediction is: {}".format(rmse(y_test, y_preds_test)))
print("Mean absolute percentage error of the prediction is: {}".format(np.mean(np.abs((y_test - y_preds_test) / y_test)) * 100))

Best alpha value is: 1000.0
R-squared of the model in training set is: 0.7119907923429403
Mean absolute percentage error of the prediction is: 15.124058726878303
-----Test set statistics-----
R-squared of the model in test set is: 0.7691048490878832
Mean absolute error of the prediction is: 24478.761543359058
Mean squared error of the prediction is: 1550161716.516951
Root mean squared error of the prediction is: 39372.093118310986
Mean absolute percentage error of the prediction is: 13.97265978988868


In [70]:
# Fitting an ElasticNet regression model with cross valivation.
elasticnet_cv = ElasticNetCV(alphas=alphas, cv=10)

elasticnet_cv.fit(X_train, y_train)

# We are making predictions here
y_preds_train = elasticnet_cv.predict(X_train)
y_preds_test = elasticnet_cv.predict(X_test)

print("Best alpha value is: {}".format(elasticnet_cv.alpha_))
print("R-squared of the model in training set is: {}".format(elasticnet_cv.score(X_train, y_train)))
print("Mean absolute percentage error of the prediction is: {}".format(np.mean(np.abs((y_train - y_preds_train) / y_train)) * 100))
print("-----Test set statistics-----")
print("R-squared of the model in test set is: {}".format(elasticnet_cv.score(X_test, y_test)))
print("Mean absolute error of the prediction is: {}".format(mean_absolute_error(y_test, y_preds_test)))
print("Mean squared error of the prediction is: {}".format(mse(y_test, y_preds_test)))
print("Root mean squared error of the prediction is: {}".format(rmse(y_test, y_preds_test)))
print("Mean absolute percentage error of the prediction is: {}".format(np.mean(np.abs((y_test - y_preds_test) / y_test)) * 100))

Best alpha value is: 0.1
R-squared of the model in training set is: 0.7180249492472212
Mean absolute percentage error of the prediction is: 14.989528523696945
-----Test set statistics-----
R-squared of the model in test set is: 0.7696691783101346
Mean absolute error of the prediction is: 24501.14129229679
Mean squared error of the prediction is: 1546372977.1155813
Root mean squared error of the prediction is: 39323.949154625625
Mean absolute percentage error of the prediction is: 13.980564974444883


> ### Each of the three regularization regression models outperformed regular OLS in both $R^2$ and *MAPE*. Interestingly, they all had better performance on their test set than their training set, unlike the OLS model with ElasticNet performing the best.